# Nomad File Uploader

In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import ipywidgets as widgets
from ipyvuetify.extra import FileInput
import hashlib
from IPython.display import display, Markdown, HTML
import os
import sys
import pandas as pd
import numpy as np
import requests
import time
sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_nomad_ids_of_entry, get_sample_description

from IPython.display import HTML, display
display(HTML("<script>document.title='NOMAD File Uploader'</script>"))

url_base = "http://localhost" # ⚠️⚠️⚠️⚠️⚠️⚠️ add your deployment url here ⚠️⚠️⚠️⚠️⚠️⚠️
url = f"{url_base}/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

# Main buttons and outputs
load_button = widgets.Button(description="Load Data")
upload_and_process = widgets.Button(description="Upload and process Data (not reversible)", layout=widgets.Layout(width='800px', height='80px'))
unrecognized_files_widget = widgets.HTML(value="",layout=widgets.Layout(width='500px'))
out = widgets.Output()
out2 = widgets.Output()

# Create a container for the upload_and_process button
upload_button_container = widgets.Output()

# Add the missing search_field widget
search_field = widgets.Text(
    value='',
    placeholder='Search batches...',
    description='Search:',
    disabled=False,
    layout=widgets.Layout(width='400px')
)

# Batch selection widgets
batch_ids_list_tmp = list(get_batch_ids(url, token))
time.sleep(0.2)
batch_ids_list = []
for b in batch_ids_list_tmp:
    if "_".join(b.split("_")[:-1]) in batch_ids_list_tmp:
        continue
    batch_ids_list.append(b)

batch_ids = widgets.Select(options=batch_ids_list, description='Batches', layout=widgets.Layout(width='800px', height='200px')) 

# File upload widget using ipyvuetify
#file_input = FileInput(
#    multiple=True,
#    accept='*',
#    show_progress=True,
#    v_model=[],
#    label='Click to add files',
#    style_='display: none;'  # Hide the entire widget
#)
file_input = FileInput(
    multiple=True,
    label='Click to add files',
    v_model=[],
    # Use the 'selection' slot to control what is displayed
    v_slots=[{
        'name': 'selection',
        # This template will display nothing, effectively hiding the file list
        'children': ''
        # Alternatively, show just a file count like this:
        # 'children': '{{ len(props.files) }} file(s) selected'
    }]
)

# Hide the file chips using custom CSS
display(HTML("""
<style>
    .v-chip {
        display: none !important;
    }
    .v-file-input__text {
        display: none !important;
    }
</style>
"""))

# Create a custom upload button and file count display
#upload_button_custom = widgets.Button(
#    description="Select Files",
#    icon='upload',
#    layout=widgets.Layout(width='150px')
#)

file_count_display = widgets.HTML(value="<i>No files selected</i>")

def trigger_file_input(b):
    file_input.fire_event('click', {})
    
#upload_button_custom.on_click(trigger_file_input)

file_selector = widgets.SelectMultiple(
    options=[],
    description='Files:',
    disabled=False,
    layout=widgets.Layout(width='450px', height='500px')
)

# Global variables
upload_files = []
sample_id_buttons = []
output_areas = {}
sample_files_dict = {}
file_type_dict = {}  # Dictionary to store file type selections for each sample and file
selected_sample_id = None
raw_upload_data = None
uploaded_files_data = {}  # Store file name to file data mapping

def get_upload_folder(upload_id):
    for upload_folder in os.listdir("../.."):
        if upload_id not in upload_folder:
            continue
        return upload_folder

def process_upload(upload_id):
    response = requests.post(f'{url}/uploads/{upload_id}/action/process', headers={'Authorization': f'Bearer {token}'})
    while True:
        time.sleep(2)
        response = requests.get(f'{url}/uploads/{upload_id}', headers={'Authorization': f'Bearer {token}'})
        if not response.json()["data"]["process_running"]:
            break
        with out2:
            print('processing')

def extract_filenames_from_vuetify(file_data_list):
    """Extract filenames from ipyvuetify FileInput data"""
    filenames = []
    for file_data in file_data_list:
        if isinstance(file_data, dict) and 'name' in file_data:
            filenames.append(file_data['name'])
    return filenames

def on_file_input_change(change):
    """Handle file input change for ipyvuetify FileInput"""
    try:
        global raw_upload_data, uploaded_files_data
        
        # Get the file data from the change event
        file_data_list = file_input.get_files()
        
        if not file_data_list:
            return
            
        # Extract filenames
        filenames = extract_filenames_from_vuetify(file_data_list)
        
        # Store file data for later use
        uploaded_files_data = {}
        for file_data in file_data_list:
            if isinstance(file_data, dict) and 'name' in file_data:
                uploaded_files_data[file_data['name']] = file_data
        
        # Prefilter files
        recognized_files = []
        unrecognized_files = []
        files_with_dots = []
        
        for filename in filenames:
            # Check for dots in filename (excluding the extension)
            base_name = os.path.splitext(filename)[0]
            if '.' in base_name:
                files_with_dots.append(filename)
            
            # Continue with recognition logic
            filename_lower = filename.lower()
            if any(keyword in filename_lower for keyword in ['jv', 'eqe', 'mppt', 'sem', 'xrd', 'xps']):
                recognized_files.append(filename)
            else:
                unrecognized_files.append(filename)
        
        # Update file selector
        file_selector.options = sorted(recognized_files + unrecognized_files)

        file_count = len(filenames)
        if file_count > 0:
            file_count_display.value = f"<b>{file_count} files selected</b>"
        else:
            file_count_display.value = "<i>No files selected</i>"
        
        with out2:
            out2.clear_output()
            print(f"Uploaded {len(filenames)} files. {len(recognized_files)} recognized files added to selector.")
            
            # Display unrecognized files warning
            if unrecognized_files:
                unrecognized_html = "<div style='color: #d9534f; padding: 10px; border: 1px solid #d9534f; border-radius: 5px; margin-top: 10px;'>"
                unrecognized_html += "<p><strong>The following files were not recognized:</strong></p>"
                unrecognized_html += "<p>Make sure to select the correct type for them: jv, eqe, mppt, sem, xrd, xps</p>"
                unrecognized_html += "<ul style='max-height: 150px; overflow-y: auto;'>"
                for file in unrecognized_files:
                    unrecognized_html += f"<li>{file}</li>"
                unrecognized_html += "</ul></div>"
                display(widgets.HTML(unrecognized_html))
            
            # Display files with dots warning
            if files_with_dots:
                dots_html = "<div style='color: #f0ad4e; padding: 10px; border: 1px solid #f0ad4e; border-radius: 5px; margin-top: 10px;'>"
                dots_html += "<p><strong>Warning: The following filenames contain periods (.) which may cause issues:</strong></p>"
                dots_html += "<p>It's recommended to use underscores (_) instead of periods in filenames.</p>"
                dots_html += "<ul style='max-height: 150px; overflow-y: auto;'>"
                for file in files_with_dots:
                    dots_html += f"<li>{file}</li>"
                dots_html += "</ul></div>"
                display(widgets.HTML(dots_html))
                
    except Exception as e:
        with out2:
            out2.clear_output()
            print(f"Error processing upload: {e}")

def on_selection_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected = change['new']
        with out2:
            out2.clear_output()
            if selected:
                print(f"Selected files: {', '.join(selected)}")
            else:
                print("No files selected")

def on_remove_button_click(sample_id, sample_select):
    def handle_click(b):
        global sample_files_dict

        # Get selected files to remove
        selected_files = list(sample_select.value)

        if selected_files:
            # Remove selected files from the sample's list
            sample_files_dict[sample_id] = [f for f in sample_files_dict[sample_id] if f not in selected_files]

            # Update the SelectMultiple widget
            sample_select.options = sample_files_dict[sample_id]

            # Add the removed files back to the file_selector
            file_selector.options = list(file_selector.options) + selected_files

            with out2:
                out2.clear_output()
                print(f"Removed files from {sample_id}:")
                print(f"Remaining files: {sample_files_dict[sample_id]}")

    return handle_click

def on_sample_button_click(sample_id, sample_select):
    def handle_click(b):
        global selected_sample_id, sample_files_dict
        selected_sample_id = sample_id
        selected_files = list(file_selector.value)

        if selected_files:
            # Update the dictionary by appending the selected files
            sample_files_dict[sample_id].extend(selected_files)

            # Update the SelectMultiple widget with the current files
            sample_select.options = sample_files_dict[sample_id]

            # Remove selected files from SelectMultiple widget
            file_selector.options = [f for f in file_selector.options if f not in selected_files]

            # Print the updated dictionary for verification
            with out2:
                out2.clear_output()
                print(f"Updated files for {sample_id}:")
                print(sample_files_dict[sample_id])
    return handle_click

def on_sample_button_first_click(sample_id, output_area):
    def handle_first_click(b):
        global sample_files_dict, file_type_dict

        # Create SelectMultiple widget for this sample
        sample_select = widgets.SelectMultiple(
            options=sample_files_dict[sample_id],
            description='',
            disabled=False,
            layout=widgets.Layout(width='230px', height='100px')
        )

        # Create Add button (for subsequent additions)
        add_button = widgets.Button(
            description="Add Files",
            layout=widgets.Layout(width='80px', padding='0px', margin='2px',  overflow='hidden')
        )
        
        # Create Remove button
        remove_button = widgets.Button(
            description="Remove",
            layout=widgets.Layout(width='80px', padding='0px', margin='2px', overflow='hidden')
        )

        # Create container for file type dropdowns
        file_type_container = widgets.VBox([], layout=widgets.Layout(margin='0', padding='0'))

        # Define measurement types
        measurement_types = ['jv', 'eqe', 'mppt', 'sem', 'xrd', 'hy', 'xps']

        # Function to update the file type dropdowns
        def update_file_types():
            # Initialize file_type_dict entries for this sample if they don't exist
            if sample_id not in file_type_dict:
                file_type_dict[sample_id] = {}

            # Create dropdown widgets for each file
            dropdown_widgets = []
            for file_name in sample_files_dict[sample_id]:
                # Determine default type based on filename
                default_type = 'hy'  # Default
                file_lower = file_name.lower()
                for mtype in ['jv', 'eqe', 'mppt', 'sem', 'xrd', 'xps']:
                    if mtype in file_lower:
                        default_type = mtype
                        break

                # Initialize in dictionary if not present
                if file_name not in file_type_dict[sample_id]:
                    file_type_dict[sample_id][file_name] = default_type

                # Create dropdown for this file
                dropdown = widgets.Dropdown(
                    options=measurement_types,
                    value=file_type_dict[sample_id][file_name],
                    description='',
                    layout=widgets.Layout(width='80px', height='22px')
                )

                # Create observer to update dictionary when dropdown changes
                def make_observer(fname):
                    def observer(change):
                        file_type_dict[sample_id][fname] = change['new']
                    return observer

                dropdown.observe(make_observer(file_name), names='value')

                # Create row with filename and dropdown
                truncated_name = file_name[:15] + '...' if len(file_name) > 20 else file_name
                row = widgets.HBox([
                    dropdown,
                    widgets.HTML(f"<div style='width:150px; font-size:0.9em; overflow:hidden'>{truncated_name}</div>"),
                ], layout=widgets.Layout(margin='0', padding='0', height='25px'))

                dropdown_widgets.append(row)

            # Update the container with all dropdown widgets
            file_type_container.children = tuple(dropdown_widgets)

        # Set up callbacks
        add_button.on_click(on_sample_button_click(sample_id, sample_select))
        remove_button.on_click(on_remove_button_click(sample_id, sample_select))

        # Observer for when files are added/removed
        def on_options_change(change):
            update_file_types()

        sample_select.observe(on_options_change, names='options')

        # Initial update of file types
        update_file_types()

        # Display the widgets in the output area
        with output_area:
            output_area.clear_output()

            # Create a horizontal layout with the buttons and sample_select on the left
            # and the file type container on the right
            display(widgets.HBox([
                # Left side: buttons and sample_select
                widgets.HBox([
                    widgets.VBox([
                        add_button,
                        remove_button,
                    ]),
                    sample_select
                ]),

                # Right side: file type container with a header
                widgets.VBox([
                    widgets.HTML("<small style='margin-bottom:5px'><b>Recognized Type:</b></small>"),
                    file_type_container
                ], layout=widgets.Layout(margin='0 0 0 15px'))  # Add left margin for spacing
            ]))

        # Also handle the initial file transfer
        selected_files = list(file_selector.value)
        if selected_files:
            # Update the dictionary by appending the selected files
            sample_files_dict[sample_id].extend(selected_files)

            # Update the SelectMultiple widget
            sample_select.options = sample_files_dict[sample_id]

            # Remove selected files from file_selector
            file_selector.options = [f for f in file_selector.options if f not in selected_files]

            # Update file types after adding files
            update_file_types()

            with out2:
                out2.clear_output()
                print(f"Added files to {sample_id}:")
                print(sample_files_dict[sample_id])

    return handle_first_click

def on_upload_file(b):
    global sample_files_dict, uploaded_files_data, file_type_dict
    upload_ids = []

    # Process files for each sample ID in the dictionary
    for sample_id, file_names in sample_files_dict.items():
        if not file_names:  # Skip if no files assigned to this sample
            continue

        # Get upload ID for this sample
        entry_id, upload_id = get_nomad_ids_of_entry(url, token, sample_id)
        time.sleep(0.2)
        upload_ids.append(upload_id)
        upload_folder = get_upload_folder(upload_id)

        # Process each file assigned to this sample ID
        for file_name in file_names:
            # Get file data from our stored mapping
            if file_name not in uploaded_files_data:
                with out2:
                    print(f"Could not find data for file: {file_name}")
                continue
                
            file_data = uploaded_files_data[file_name]

            # Get file data from our stored mapping
            if file_name not in uploaded_files_data:
                with out2:
                    print(f"Could not find data for file: {file_name}")
                continue
                
            file_data = uploaded_files_data[file_name]
            
            # Debug: print the structure of file_data
            with out2:
                #print(f"Debug - file_data keys for {file_name}: {list(file_data.keys())}")
                #print(f"Debug - file_data type: {type(file_data)}")
                # Print first 100 chars of each key's value
                for key in file_data.keys():
                    value_preview = str(file_data[key])[:100] if file_data[key] else "None"
                    #print(f"Debug - {key}: {value_preview}...")
            
            # Get the file content from the ClientSideFile object
            if 'file_obj' in file_data:
                file_obj = file_data['file_obj']
                # Read the content from the ClientSideFile object
                file_content = file_obj.read()
            else:
                with out2:
                    print(f"No data found in file: {file_name}")
                continue

            # Split filename for processing
            file_name_parts = file_name.split(".")
            file_type = file_name_parts[-1]
            file_name_old = ".".join(file_name_parts[:-1])

            # Get measurement type from dropdown selection if available
            if sample_id in file_type_dict and file_name in file_type_dict[sample_id]:
                measurement_type = file_type_dict[sample_id][file_name]
            else:
                # Fallback to determining type based on file name
                measurement_type = "hy"  # Default
                file_lower = file_name.lower()
                if "jv" in file_lower:
                    measurement_type = "jv"
                elif "mppt" in file_lower:
                    measurement_type = "mppt"
                elif "eqe" in file_lower:
                    measurement_type = "eqe"
                elif "sem" in file_lower:
                    measurement_type = "sem"
                elif "xrd" in file_lower:
                    measurement_type = "xrd"
                elif "xps" in file_lower:
                    measurement_type = "xps"

            # Create new file name with appropriate measurement type
            new_file_name = f"{sample_id}.{file_name_old}.{measurement_type}.{file_type}"

            with out2:
                print(f"Writing file: {new_file_name}")
                if not upload_folder:
                    display(widgets.HTML(f'<p style="color:red;">No write access to upload {upload_id}!!!</p>'))
                    display(widgets.HTML(f'<p style="color:red;">Check access and restart voila!!</p>'))
                    continue               
                else: 
                    print(upload_folder)

            # Write the file
            with open(f"../../{upload_folder}/{new_file_name}", "wb") as f:
                f.write(file_content)

    # Process uploads
    for upload_id in set(upload_ids):
        process_upload(upload_id) # process uploads twice!
        time.sleep(1)
        process_upload(upload_id)

    with out2:
        print("Upload processed")
        print(f"Processed {len(upload_ids)} sample IDs with assigned files")
        
def on_load_button_clicked(b):
    global out, sample_id_buttons, output_areas, sample_files_dict
    out.clear_output()

    # Clear previous buttons and output areas
    sample_id_buttons = []
    output_areas = {}  # Change to dictionary to track by sample_id

    # Get sample IDs for the selected batch
    sample_ids = get_ids_in_batch(url, token, [batch_ids.value])
    sample_descriptions = get_sample_description(url, token, sample_ids)
    print(len(sample_descriptions),sample_descriptions)
    print(len(sample_ids),sample_ids)
    time.sleep(0.2)

    # Initialize the dictionary with empty arrays for each sample ID
    sample_files_dict = {sample_id: [] for sample_id in sample_ids}

    # Create a container for each sample ID with initially just the sample button
    for sample_id in sample_ids:  
        description = sample_descriptions[sample_id] if sample_id in sample_descriptions else ''
        sample_button = widgets.Button(  
            description=sample_id+" ["+description+"]",  
            layout=widgets.Layout(width='400px', height="40px")  
        )

        # Create output area to hold the SelectMultiple and Remove button
        output_area = widgets.Output(layout=widgets.Layout(height='auto', min_height='0px'))
        output_areas[sample_id] = output_area

        # Set up callback for the sample button
        sample_button.on_click(on_sample_button_first_click(sample_id, output_area))

        # Add to list
        sample_id_buttons.append(widgets.VBox([
            sample_button,
            output_area
        ], layout=widgets.Layout(
                    flex='0 0 auto',  # Don't shrink, don't grow, use natural size
            margin='2px 0'    # Small margin between buttons
        )))

    # Create the left panel (uploader section) with fixed layout
    left_panel = widgets.VBox([
        widgets.HBox([file_count_display]),
        file_input,  # Keep it but hidden
        widgets.HTML("<div style='margin-bottom:5px; font-style:italic; color:#555;'>Select files from the list below, then click on a sample ID to assign them</div>"),
        file_selector,
        unrecognized_files_widget
    ], layout=widgets.Layout(
        width='530px',
        height='800px',
        padding='10px',
        border='1px solid #ddd',
        margin='5px'
    ))
    
    # Create the right panel (sample buttons) with scrollable layout
    right_panel = widgets.VBox(
        sample_id_buttons,
        layout=widgets.Layout(
            width='600px',
            height='600px',
            overflow='scroll',  # Vertical scroll only
            padding='10px',
            border='1px solid #ddd',
            margin='5px'
        )
    )

    # Display the sample ID buttons
    with out:
        display(widgets.HTML("<h3>Select a Sample ID and Upload Files</h3>"))
        display(widgets.HBox([
            left_panel,
            right_panel
        ], layout=widgets.Layout(
            width='100%',
            align_items='flex-start'  # Align panels to the top
        )))

    # Show the upload_and_process button
    with upload_button_container:
        upload_button_container.clear_output()
        display(upload_and_process)
    
def on_search_field_change(change):
    """Automatically filter batch_ids when search_field value changes"""
    if change['type'] == 'change' and change['name'] == 'value':
        search_term = change['new'].lower()
        
        if not search_term:  # If search field is empty, show all batches
            batch_ids.options = batch_ids_list
        else:
            # Filter batches based on search term
            batch_ids.options = [d for d in batch_ids_list if search_term in d.lower()]
    
# Register callbacks
load_button.on_click(on_load_button_clicked)
upload_and_process.on_click(on_upload_file)
file_input.observe(on_file_input_change, names='file_info')  # Observe v_model changes
file_selector.observe(on_selection_change, names='value')
search_field.observe(on_search_field_change, names='value')

# Display the main interface
display(widgets.VBox([
    search_field,
    batch_ids,
    load_button,
    out,
    upload_button_container,
    out2
]))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Create the guide content as HTML
guide_content = """
<div style="padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9; margin: 10px 0;">
<h2>Quick Start Guide</h2>

<h3>1. <strong>Search and Select Batch</strong></h3>
<ul>
<li>Use the search field to filter available batches by name</li>
<li>Select your target batch from the dropdown list</li>
<li>Click <strong>"Load Data"</strong> to retrieve sample IDs for that batch</li>
</ul>

<h3>2. <strong>Upload Files</strong></h3>
<ul>
<li>Click <strong>"Choose Files"</strong> to select your measurement data files</li>
<li>The system will automatically recognize common file types:
  <ul>
    <li><strong>JV</strong>: Current-voltage measurements</li>
    <li><strong>EQE</strong>: External quantum efficiency data</li>
    <li><strong>MPPT</strong>: Maximum power point tracking</li>
    <li><strong>SEM</strong>: Scanning electron microscopy images</li>
    <li><strong>XRD</strong>: X-ray diffraction patterns</li>
    <li><strong>XPS</strong>: X-ray photoelectron spectroscopy</li>
  </ul>
</li>
<li>Unrecognized files will be flagged for manual type assignment</li>
</ul>

<h3>3. <strong>Assign Files to Samples</strong></h3>
<ul>
<li>Select files from the file list (use Ctrl/Cmd+click for multiple selection)</li>
<li>Click on a sample ID button to assign selected files to that sample</li>
<li>Each sample will show:
  <ul>
    <li>Assigned files list</li>
    <li>File type dropdowns for manual correction if needed</li>
    <li>Add/Remove buttons for file management</li>
  </ul>
</li>
</ul>

<h3>4. <strong>Verify and Process</strong></h3>
<ul>
<li>Review file assignments and types for each sample</li>
<li>Adjust file types using the dropdown menus if the automatic detection was incorrect</li>
<li>Click <strong>"Upload and process Data"</strong> when ready (⚠️ <strong>This action is not reversible</strong>)</li>
</ul>

<hr>

<h2>File Naming Conventions</h2>
<p>The system will rename your files according to NOMAD standards:</p>
<pre style="background-color: #f0f0f0; padding: 10px; border-radius: 3px;">{sample_id}.{original_filename}.{measurement_type}.{extension}</pre>

<p><strong>Example:</strong></p>
<ul>
<li>Original: <code>measurement_data.txt</code></li>
<li>Renamed: <code>SAMPLE_001.measurement_data.jv.txt</code></li>
</ul>

<hr>

<h2>Supported Measurement Types</h2>
<table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
<tr style="background-color: #f2f2f2;">
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;"><strong>Type</strong></th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;"><strong>Description</strong></th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;"><strong>Common Keywords</strong></th>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>jv</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">Current-Voltage curves</td>
<td style="border: 1px solid #ddd; padding: 8px;">jv, iv, current, voltage</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>eqe</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">External Quantum Efficiency</td>
<td style="border: 1px solid #ddd; padding: 8px;">eqe, quantum, efficiency</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>mppt</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">Maximum Power Point Tracking</td>
<td style="border: 1px solid #ddd; padding: 8px;">mppt, tracking, power</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>sem</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">Scanning Electron Microscopy</td>
<td style="border: 1px solid #ddd; padding: 8px;">sem, microscopy, image</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>xrd</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">X-Ray Diffraction</td>
<td style="border: 1px solid #ddd; padding: 8px;">xrd, diffraction, crystal</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>xps</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">X-ray photoelectron spectroscopy</td>
<td style="border: 1px solid #ddd; padding: 8px;">xps, nups, he-ups</td>
</tr>
<tr>
<td style="border: 1px solid #ddd; padding: 8px;"><strong>hy</strong></td>
<td style="border: 1px solid #ddd; padding: 8px;">Hybrid/Other measurements</td>
<td style="border: 1px solid #ddd; padding: 8px;">Default for unrecognized files</td>
</tr>
</table>
</div>
"""

# Create the guide button and collapsible content
guide_button = widgets.Button(
    description="📖 Show Guide",
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px')
)

guide_output = widgets.Output()
guide_visible = False

def toggle_guide(b):
    global guide_visible
    with guide_output:
        guide_output.clear_output()
        if not guide_visible:
            # Show the guide
            display(HTML(guide_content))
            guide_button.description = "📖 Hide Guide"
            guide_button.button_style = 'warning'
            guide_visible = True
        else:
            # Hide the guide
            guide_button.description = "📖 Show Guide"
            guide_button.button_style = 'info'
            guide_visible = False

guide_button.on_click(toggle_guide)

# Display the guide section
display(widgets.VBox([
    guide_button,
    guide_output
], layout=widgets.Layout(margin='0 0 20px 0')))